In [17]:
import os
import sys

# Ensure the current directory is in the Python path
sys.path.append('/Users/iyongjeong/WORK/dacon/clf/')
from loader import ImageDataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights, resnet18, ResNet18_Weights

In [13]:
csv_dir = '/Users/iyongjeong/WORK/dacon/clf/data/train_csv/train_file_list.csv'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageDataset(csv_dir=csv_dir, transform=transform)
dataset.load_data()

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for images, labels in dataloader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [18]:
# fine tunning
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/iyongjeong/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [19]:
def train(model, dataloader, epochs=10, learning_rate=0.001):
    import torch
    import torch.optim as optim
    import torch.nn as nn

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")

In [ ]:
# train debug
train(model, dataloader, epochs=1, learning_rate=0.001)